In [343]:
import csv
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')
import seaborn as sns
sns.set()
sns.set_context("talk")
import re
import seaborn as sns
from IPython.display import display, Latex, Markdown
#from client.api.notebook import Notebook
#ok = Notebook('proj1.ok')

In [344]:
#this vader_lexicon file comes from UC Berkley experiment for sentiment valuation and scoring
#src link:


sent = pd.read_csv('data/vader_lexicon.txt', sep='\t', header=None)
sent = sent.drop([2, 3], 1)
sent.index.rename('token', inplace=True)
sent.rename(columns={1: 'polarity'}, inplace=True)
sent.head()
#print(len(sent))

,0,polarity
token,,
0,$:,-1.5
1,%),-0.4
2,%-),-1.5
3,&-:,-0.4
4,&:,-0.7


In [345]:
#cut of datetime
#check the elect date and the inauguration date
from datetime import datetime

ELEC_DATE = datetime(2016, 11, 8)
INAUG_DATE = datetime(2017, 1, 20)

In [346]:
# when running the modeling test each time we can skip the on-demand tweet reading and use the save file
# each time you want to refresh the tweet data
# we can use the >>>>>>>>>>>>>>ReadTrumpTweetOnDemand.ipynb<<<<<<<<<<< notebook in the same folder 
# to read and save the file
# for trump in office data
# we need to use the datetime defined above to filter 

trump_tweets = pd.read_csv('data/trump-tweet-in-office.csv')
trump_tweets['created_at'] =  pd.to_datetime(trump_tweets['created_at']).dt.date
trump_tweets.head()

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str
0,Twitter for iPhone,RT @RepLeeZeldin: This impeachment inquiry has...,2019-10-25,8819,0,True,1187551862616141824
1,Twitter for iPhone,RT @RepDougCollins: Democrats have no concern ...,2019-10-25,13713,0,True,1187551461808492544
2,Twitter for iPhone,RT @RepGosar: The only reason Adam Schiff is c...,2019-10-25,10961,0,True,1187551451775750144
3,Twitter for iPhone,RT @RepArrington: Let's call this sham what it...,2019-10-25,4547,0,True,1187551305281212416
4,Twitter for iPhone,RT @RepMattGaetz: Democrats are trying to use ...,2019-10-25,10491,0,True,1187551256371515392


In [347]:
trump = trump_tweets.copy()
trump['id']=trump['id_str']
trump = trump.set_index(['id'])

trump.head()

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str
id,,,,,,,
1187551862616141824,Twitter for iPhone,RT @RepLeeZeldin: This impeachment inquiry has...,2019-10-25,8819,0,True,1187551862616141824
1187551461808492544,Twitter for iPhone,RT @RepDougCollins: Democrats have no concern ...,2019-10-25,13713,0,True,1187551461808492544
1187551451775750144,Twitter for iPhone,RT @RepGosar: The only reason Adam Schiff is c...,2019-10-25,10961,0,True,1187551451775750144
1187551305281212416,Twitter for iPhone,RT @RepArrington: Let's call this sham what it...,2019-10-25,4547,0,True,1187551305281212416
1187551256371515392,Twitter for iPhone,RT @RepMattGaetz: Democrats are trying to use ...,2019-10-25,10491,0,True,1187551256371515392


In [348]:
#remove the puncuations from the tweets

repl_punc = trump['text'].str.replace(r"[?|,|$|!|&|.|#|^|;|:|/|\"]", ' ')
replacequotes = repl_punc.str.replace(r'^"', '')
#replacequotes = repl_punc.strip('"')


In [349]:
# Save your regex in punct_re
punct_re = r"[^\w\s]"
#[?|,|$|!|&|.|#|^|;|:|/|\"]
trump['no_punc'] = trump['text'].str.replace(punct_re, " ")

In [350]:
trumpcopy = trump.copy()
changed_punc = trumpcopy['no_punc'].str.replace('\s+', ' ')
trumpcopy['no_punc'] = changed_punc
splt = trumpcopy['no_punc'].str.split(expand=True).stack().reset_index().set_index('id')
splt.columns = ['num', 'word']


In [351]:
splt.head(10)


,num,word
id,,
1187551862616141824,0,RT
1187551862616141824,1,RepLeeZeldin
1187551862616141824,2,This
1187551862616141824,3,impeachment
1187551862616141824,4,inquiry
1187551862616141824,5,has
1187551862616141824,6,produced
1187551862616141824,7,NOTHING
1187551862616141824,8,to


In [352]:
tidy_format = splt
newt = tidy_format.reset_index()
newsent = sent.reset_index().rename(columns = {'token': 'word'})

In [353]:
display(newt.head())
display(newsent.head())
#display(newt.describe())
#display(newsent.describe())


,id,num,word
0,1187551862616141824,0,RT
1,1187551862616141824,1,RepLeeZeldin
2,1187551862616141824,2,This
3,1187551862616141824,3,impeachment
4,1187551862616141824,4,inquiry


,word,0,polarity
0,0,$:,-1.5
1,1,%),-0.4
2,2,%-),-1.5
3,3,&-:,-0.4
4,4,&:,-0.7


In [354]:
newt = tidy_format.reset_index()
#newt = newt.drop('num')
newt1 = newt[['id', 'word']].copy()
newsent = sent.reset_index().rename(columns = {0: 'word'})
newsent1 =newsent[['polarity', 'word']].copy()
display(newt1.head())
display(newsent1.head(10))

,id,word
0,1187551862616141824,RT
1,1187551862616141824,RepLeeZeldin
2,1187551862616141824,This
3,1187551862616141824,impeachment
4,1187551862616141824,inquiry


,polarity,word
0,-1.5,$:
1,-0.4,%)
2,-1.5,%-)
3,-0.4,&-:
4,-0.7,&:
5,1.6,( '}{' )
6,-0.9,(%
7,2.2,('-:
8,2.3,(':
9,2.1,((-:


In [355]:
polar = pd.merge(newt1, newsent1, how='outer', on= 'word')
newpol = polar.groupby('id').sum()
###newpol = newpol.drop('num', 1).fillna(0)
newpol = newpol.fillna(0)

In [356]:
polarities = newpol
maybpol = polarities.reset_index()
maybtrump = trump.reset_index()

In [357]:
display(maybpol.head())
display(maybtrump.head())

,id,polarity
0,8.222424e+17,1.7
1,8.222987e+17,2.7
2,8.224214e+17,-1.0
3,8.225018e+17,1.7
4,8.225019e+17,1.4


,id,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str,no_punc
0,1187551862616141824,Twitter for iPhone,RT @RepLeeZeldin: This impeachment inquiry has...,2019-10-25,8819,0,True,1187551862616141824,RT RepLeeZeldin This impeachment inquiry has...
1,1187551461808492544,Twitter for iPhone,RT @RepDougCollins: Democrats have no concern ...,2019-10-25,13713,0,True,1187551461808492544,RT RepDougCollins Democrats have no concern ...
2,1187551451775750144,Twitter for iPhone,RT @RepGosar: The only reason Adam Schiff is c...,2019-10-25,10961,0,True,1187551451775750144,RT RepGosar The only reason Adam Schiff is c...
3,1187551305281212416,Twitter for iPhone,RT @RepArrington: Let's call this sham what it...,2019-10-25,4547,0,True,1187551305281212416,RT RepArrington Let s call this sham what it...
4,1187551256371515392,Twitter for iPhone,RT @RepMattGaetz: Democrats are trying to use ...,2019-10-25,10491,0,True,1187551256371515392,RT RepMattGaetz Democrats are trying to use ...


In [358]:
senti = pd.merge(maybpol, maybtrump, how='outer', on='id').set_index('id')


In [359]:
new_senti= senti.loc[senti['favorite_count']>=0].drop('id_str',axis=1)
print(len(maybtrump),len(new_senti))
display(new_senti.head())

11568 11568


,polarity,source,text,created_at,retweet_count,favorite_count,is_retweet,no_punc
id,,,,,,,,
8.225021e+17,3.7,Twitter for iPhone,What truly matters is not which party controls...,2017-01-20,42790.0,180394.0,False,What truly matters is not which party controls...
8.225026e+17,3.9,Twitter for iPhone,We will bring back our jobs. We will bring bac...,2017-01-20,37458.0,173821.0,False,We will bring back our jobs We will bring bac...
8.225029e+17,0.0,Twitter for iPhone,We will follow two simple rules: BUY AMERICAN ...,2017-01-20,38092.0,161358.0,False,We will follow two simple rules BUY AMERICAN ...
8.225041e+17,0.0,Twitter for iPhone,So to all Americans in every city near and far...,2017-01-20,14039.0,86467.0,False,So to all Americans in every city near and far...
8.226691e+17,2.7,Twitter for iPhone,THANK YOU for another wonderful evening in Was...,2017-01-21,52468.0,289942.0,False,THANK YOU for another wonderful evening in Was...


In [360]:
keytags_politics_partisans =['crooked hilary|joe biden|obama administration|CORRUPTION','mexican wall','immigration|election medling|gun control','Nancy Pelsi','dem\W','dem party','impeach|impeachment','Russia','Putin'
                             ,'fake news']
keytags_foreign_policys =['ceasefire zones|USA soldiers|ISIS[\W]|Endless Wars','Nato\W','Middle East','Israel','Saudi\W','iran\W','iraq\W','middel east','missle'
                          ,'japan\W','korean','Mexican[\w\W]+Wall','Syria','Rocket Man','Turkey|Turks[\W|Syria|Kurdish|Kurd[\W]']
keytags_trade_war =['tpp','currency manipulate|currency manipulation','wto','unfair[\w\W]+trade'
                    ,'trade deficits|trade deficit','trade deal|trade deals','china[\w\W]+trade','trade','china','tariff'
                    ,'american[\w\W]+manufac','american first','free fair|SMART TRADE']
keytags_trade_war_senti =['worst trade deal|STUPID TRADE','currency manipulate|currency manipulation','china[\w\W]+tariff','new tariff|increase tariff','unfair[\w\W]+trade','taken advantage|taking advantage|ripped off']


In [361]:
trade_tags_all = []
sentiments_tags_all = []
trade_tags_score = []
foreign_policys=[]
politics_partisans=[]

df_tweets = new_senti.copy()
counter =0


In [362]:

#do my own trade key tag point calculation
# regular trade key tag +1
# senti key tag +3

for i in range(len(df_tweets)) :
    #print(i)
    tagscore =0 
    senti_tag =[]
    trade_tag =[]
    fp_tag =[]
    pp_tag =[]

    tweet_text = df_tweets.iloc[i,2]
    
    #trade tag, plus 2
    counter = 0
    for tag in keytags_trade_war:
        trade_tag = trade_tag + re.findall(tag,tweet_text, re.IGNORECASE)
        if len(trade_tag)>counter:
            counter +=1
            tagscore += 2
            print('trade tag:', i, trade_tag)
    
    #centi trade tag, plus 5
    counter = 0
    for tag in keytags_trade_war_senti:
        senti_tag = senti_tag + re.findall(tag,tweet_text, re.IGNORECASE)
        if len(senti_tag)>counter:
            counter +=1
            tagscore += 5
            print('trade centi:', i, trade_tag)
            #print(tweet_text)
    
    #foreign policy tag, plus 1
    counter = 0
    for tag in keytags_foreign_policys:
        fp_tag = fp_tag + re.findall(tag,tweet_text, re.IGNORECASE)
        if len(fp_tag)>counter:
            counter +=1
            tagscore += 1
            print('foreign policy:', i, trade_tag)
    
    for tag in keytags_politics_partisans:
        pp_tag = pp_tag + re.findall(tag,tweet_text, re.IGNORECASE)

    trade_tags_score.append(tagscore)   
    trade_tags_all.append(trade_tag)
    sentiments_tags_all.append(senti_tag)
    foreign_policys.append(fp_tag)
    politics_partisans.append(pp_tag)
    
        
print(counter)

trade tag: 20 ['trade deficit']
trade tag: 20 ['trade deficit', 'trade']
trade tag: 24 ['trade deficits']
trade tag: 24 ['trade deficits', 'trade']
trade centi: 24 ['trade deficits', 'trade']
foreign policy: 33 []
foreign policy: 45 []
foreign policy: 46 []
foreign policy: 46 []
foreign policy: 47 []
foreign policy: 62 []
trade tag: 74 ['China']
trade tag: 165 ['China']
foreign policy: 167 []
foreign policy: 185 []
trade tag: 190 ['trade deficits']
trade tag: 190 ['trade deficits', 'China will be a very difficult one in that we can no longer have massive trade']
trade tag: 190 ['trade deficits', 'China will be a very difficult one in that we can no longer have massive trade', 'trade']
trade tag: 190 ['trade deficits', 'China will be a very difficult one in that we can no longer have massive trade', 'trade', 'China']
trade tag: 194 ['American Manufac']
trade tag: 197 ['trade']
trade tag: 197 ['trade', 'American manufac']
trade tag: 208 ['China']
foreign policy: 209 []
foreign policy: 21

foreign policy: 1953 []
trade tag: 1955 ['China this morning at 8:30. The primary topics will be Trade']
trade tag: 1955 ['China this morning at 8:30. The primary topics will be Trade', 'Trade']
trade tag: 1955 ['China this morning at 8:30. The primary topics will be Trade', 'Trade', 'China']
foreign policy: 1957 []
foreign policy: 1964 []
foreign policy: 1968 []
foreign policy: 1970 []
trade tag: 1971 ['trade deal']
trade tag: 1971 ['trade deal', 'trade']
trade tag: 1971 ['trade deal', 'trade', 'China']
trade tag: 1978 ['trade deal']
trade tag: 1978 ['trade deal', 'China. Nothing has happened with ZTE except as it pertains to the larger trade']
trade tag: 1978 ['trade deal', 'China. Nothing has happened with ZTE except as it pertains to the larger trade', 'trade', 'trade']
trade tag: 1978 ['trade deal', 'China. Nothing has happened with ZTE except as it pertains to the larger trade', 'trade', 'trade', 'China', 'China']
trade tag: 1978 ['trade deal', 'China. Nothing has happened with Z

trade tag: 2420 ['Trade']
foreign policy: 2435 []
trade tag: 2459 ['Trade Deal']
trade tag: 2459 ['Trade Deal', 'China market has dropped 27% in last 4months and they are talking to us. Our market is stronger than ever and will go up dramatically when these horrible Trade']
trade tag: 2459 ['Trade Deal', 'China market has dropped 27% in last 4months and they are talking to us. Our market is stronger than ever and will go up dramatically when these horrible Trade', 'Trade']
trade tag: 2459 ['Trade Deal', 'China market has dropped 27% in last 4months and they are talking to us. Our market is stronger than ever and will go up dramatically when these horrible Trade', 'Trade', 'China']
trade tag: 2459 ['Trade Deal', 'China market has dropped 27% in last 4months and they are talking to us. Our market is stronger than ever and will go up dramatically when these horrible Trade', 'Trade', 'China', 'Tariff']
trade tag: 2460 ['trade deal']
trade tag: 2460 ['trade deal', 'trade']
trade tag: 2460 [

trade tag: 3557 ['trade deal']
trade tag: 3557 ['trade deal', 'trade']
foreign policy: 3579 []
foreign policy: 3589 []
foreign policy: 3611 []
foreign policy: 3611 []
foreign policy: 3622 []
foreign policy: 3622 []
foreign policy: 3626 []
foreign policy: 3626 []
foreign policy: 3626 []
foreign policy: 3626 []
trade tag: 3629 ['China’s top trade']
trade tag: 3629 ['China’s top trade', 'trade']
trade tag: 3629 ['China’s top trade', 'trade', 'China', 'China']
trade tag: 3629 ['China’s top trade', 'trade', 'China', 'China', 'Tariff']
trade tag: 3629 ['China’s top trade', 'trade', 'China', 'China', 'Tariff']
trade centi: 3629 ['China’s top trade', 'trade', 'China', 'China', 'Tariff']
trade tag: 3630 ['China', 'China']
trade tag: 3630 ['China', 'China', 'Tariff']
trade tag: 3630 ['China', 'China', 'Tariff']
trade centi: 3630 ['China', 'China', 'Tariff']
trade tag: 3632 ['China']
foreign policy: 3633 []
foreign policy: 3635 []
foreign policy: 3635 []
foreign policy: 3663 []
foreign policy: 36

trade tag: 4454 ['China']
trade tag: 4454 ['China', 'Tariff']
trade tag: 4455 ['China', 'China']
trade tag: 4455 ['China', 'China']
foreign policy: 4472 []
foreign policy: 4472 []
trade tag: 4489 ['tariff']
foreign policy: 4491 []
trade tag: 4495 ['China']
foreign policy: 4497 []
foreign policy: 4498 []
trade tag: 4517 ['trade']
foreign policy: 4517 ['trade']
foreign policy: 4517 ['trade']
trade tag: 4543 ['Trade']
foreign policy: 4543 ['Trade']
foreign policy: 4543 ['Trade']
trade tag: 4544 ['Trade']
foreign policy: 4544 ['Trade']
foreign policy: 4544 ['Trade']
foreign policy: 4549 []
trade tag: 4551 ['trade']
foreign policy: 4553 []
foreign policy: 4557 []
foreign policy: 4557 []
foreign policy: 4557 []
foreign policy: 4558 []
trade tag: 4560 ['Trade']
foreign policy: 4560 ['Trade']
foreign policy: 4561 []
trade tag: 4562 ['WTo']
foreign policy: 4562 ['WTo']
foreign policy: 4564 []
foreign policy: 4566 []
foreign policy: 4571 []
foreign policy: 4574 []
foreign policy: 4574 []
foreign

foreign policy: 5504 []
foreign policy: 5504 []
trade tag: 5512 ['China']
trade tag: 5512 ['China', 'Tariff']
trade centi: 5512 ['China', 'Tariff']
trade tag: 5514 ['China (and many other countries) has been taking advantage of the United States on Trade']
trade tag: 5514 ['China (and many other countries) has been taking advantage of the United States on Trade', 'Trade']
trade tag: 5514 ['China (and many other countries) has been taking advantage of the United States on Trade', 'Trade', 'China', 'China']
trade tag: 5514 ['China (and many other countries) has been taking advantage of the United States on Trade', 'Trade', 'China', 'China']
trade centi: 5514 ['China (and many other countries) has been taking advantage of the United States on Trade', 'Trade', 'China', 'China']
trade tag: 5515 ['Trade']
trade tag: 5517 ['China']
trade tag: 5520 ['American energy and manufac']
trade tag: 5525 ['China']
trade tag: 5526 ['Trade']
trade tag: 5526 ['Trade', 'China']
trade tag: 5540 ['Trade Deal

trade tag: 7059 ['trade deal']
trade tag: 7059 ['trade deal', 'trade']
foreign policy: 7064 []
foreign policy: 7064 []
trade tag: 7068 ['China', 'China']
trade tag: 7068 ['China', 'China']
foreign policy: 7102 []
trade tag: 7107 ['trade']
trade tag: 7109 ['Tariff']
trade tag: 7110 ['China', 'China']
trade tag: 7110 ['China', 'China']
trade tag: 7121 ['China', 'China']
trade tag: 7121 ['China', 'China', 'Tariff']
trade tag: 7121 ['China', 'China', 'Tariff']
trade centi: 7121 ['China', 'China', 'Tariff']
foreign policy: 7193 []
foreign policy: 7193 []
trade tag: 7206 ['CHINA']
trade tag: 7209 ['China']
foreign policy: 7214 []
trade tag: 7220 ['China', 'China']
trade tag: 7220 ['China', 'China']
trade tag: 7222 ['Trade']
trade tag: 7223 ['Tariff']
trade tag: 7226 ['Tariff']
trade tag: 7227 ['Trade']
trade tag: 7255 ['Trade']
trade tag: 7255 ['Trade', 'China']
foreign policy: 7283 []
foreign policy: 7283 []
trade tag: 7286 ['Tariff']
trade tag: 7288 ['trade']
trade tag: 7289 ['tariff']
tra

trade tag: 8245 ['Tariff']
trade tag: 8248 ['China']
trade tag: 8249 ['TARIFF']
trade tag: 8250 ['China']
trade tag: 8251 ['Tariff']
trade tag: 8259 ['China']
trade tag: 8262 ['unfair trade']
trade tag: 8262 ['unfair trade', 'China’s unfair trade']
trade tag: 8262 ['unfair trade', 'China’s unfair trade', 'trade']
trade tag: 8262 ['unfair trade', 'China’s unfair trade', 'trade', 'China']
trade centi: 8262 ['unfair trade', 'China’s unfair trade', 'trade', 'China']
trade tag: 8264 ['China']
trade tag: 8270 ['TARIFF']
trade tag: 8271 ['unfair trade']
trade tag: 8271 ['unfair trade', 'China has engaged in unfair trade']
trade tag: 8271 ['unfair trade', 'China has engaged in unfair trade', 'trade']
trade tag: 8271 ['unfair trade', 'China has engaged in unfair trade', 'trade', 'China', 'China']
trade tag: 8271 ['unfair trade', 'China has engaged in unfair trade', 'trade', 'China', 'China']
trade centi: 8271 ['unfair trade', 'China has engaged in unfair trade', 'trade', 'China', 'China']
trade

trade tag: 9185 ['China', 'China', 'China']
trade tag: 9185 ['China', 'China', 'China']
trade tag: 9185 ['China', 'China', 'China']
trade tag: 9191 ['Trade Deal']
trade tag: 9191 ['Trade Deal', 'Trade']
trade tag: 9205 ['China']
trade tag: 9216 ['Trade']
trade centi: 9216 ['Trade']
foreign policy: 9216 ['Trade']
trade tag: 9218 ['Trade']
foreign policy: 9228 []
foreign policy: 9251 []
trade tag: 9252 ['Trade']
trade tag: 9261 ['China']
foreign policy: 9261 ['China']
trade tag: 9268 ['Trade Deal']
trade tag: 9268 ['Trade Deal', 'Trade']
foreign policy: 9365 []
foreign policy: 9391 []
foreign policy: 9397 []
foreign policy: 9398 []
foreign policy: 9398 []
foreign policy: 9398 []
trade tag: 9399 ['Trade']
foreign policy: 9410 []
foreign policy: 9413 []
foreign policy: 9429 []
foreign policy: 9430 []
trade tag: 9479 ['Trade Deal']
trade tag: 9479 ['Trade Deal', 'Trade']
trade tag: 9482 ['Trade']
trade tag: 9488 ['Trade']
trade tag: 9490 ['Trade Deal']
trade tag: 9490 ['Trade Deal', 'Trade'

trade tag: 10281 ['Trade']
foreign policy: 10281 ['Trade']
foreign policy: 10289 []
foreign policy: 10293 []
trade tag: 10294 ['Trade']
foreign policy: 10294 ['Trade']
trade tag: 10298 ['China']
foreign policy: 10299 []
trade tag: 10310 ['trade']
trade tag: 10310 ['trade', 'China']
foreign policy: 10310 ['trade', 'China']
trade tag: 10312 ['China']
trade tag: 10312 ['China', 'tariff']
trade centi: 10312 ['China', 'tariff']
foreign policy: 10315 []
foreign policy: 10315 []
foreign policy: 10316 []
trade tag: 10317 ['China will take down its Trade']
trade tag: 10317 ['China will take down its Trade', 'trade', 'Trade']
trade tag: 10317 ['China will take down its Trade', 'trade', 'Trade', 'China']
trade tag: 10317 ['China will take down its Trade', 'trade', 'Trade', 'China']
trade tag: 10322 ['WTO']
trade tag: 10322 ['WTO', 'China which is a great economic power is considered a Developing Nation within the World Trade']
trade tag: 10322 ['WTO', 'China which is a great economic power is con

In [363]:
df_tweets_padded = df_tweets.copy()
df_tweets_padded['trade_tags_score']=trade_tags_score
df_tweets_padded['tradetag']=trade_tags_all
df_tweets_padded['senti_tradetag']=sentiments_tags_all
df_tweets_padded['foreign_policys_tag']=foreign_policys
df_tweets_padded['politics_partisans_tag']=politics_partisans

display(df_tweets_padded.head(200))
df_tweets_padded['trump_tweet_cnt'] =[1 if reFlag ==False else 0 for reFlag in df_tweets_padded['is_retweet'].values]
df_tweets_padded['trump_retweet_cnt'] =[0 if  reFlag ==False else 1 for reFlag in df_tweets_padded['is_retweet'].values]


,polarity,source,text,created_at,retweet_count,favorite_count,is_retweet,no_punc,trade_tags_score,tradetag,senti_tradetag,foreign_policys_tag,politics_partisans_tag
id,,,,,,,,,,,,,
8.225021e+17,3.7,Twitter for iPhone,What truly matters is not which party controls...,2017-01-20,42790.0,180394.0,False,What truly matters is not which party controls...,0,[],[],[],[]
8.225026e+17,3.9,Twitter for iPhone,We will bring back our jobs. We will bring bac...,2017-01-20,37458.0,173821.0,False,We will bring back our jobs We will bring bac...,0,[],[],[],[]
8.225029e+17,0.0,Twitter for iPhone,We will follow two simple rules: BUY AMERICAN ...,2017-01-20,38092.0,161358.0,False,We will follow two simple rules BUY AMERICAN ...,0,[],[],[],[]
8.225041e+17,0.0,Twitter for iPhone,So to all Americans in every city near and far...,2017-01-20,14039.0,86467.0,False,So to all Americans in every city near and far...,0,[],[],[],[]
8.226691e+17,2.7,Twitter for iPhone,THANK YOU for another wonderful evening in Was...,2017-01-21,52468.0,289942.0,False,THANK YOU for another wonderful evening in Was...,0,[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8.481539e+17,-1.2,Twitter for iPhone,When will Sleepy Eyes Chuck Todd and @NBCNews ...,2017-04-01,21797.0,78221.0,False,When will Sleepy Eyes Chuck Todd and NBCNews ...,0,[],[],[],[Russia]
8.481586e+17,-3.9,Twitter for iPhone,It is the same Fake News Media that said there...,2017-04-01,22380.0,86222.0,False,It is the same Fake News Media that said there...,0,[],[],[],"[Russia, Fake News]"
8.481971e+17,0.0,Twitter for iPhone,RT @DanScavino: .@POTUS @realDonaldTrump signs...,2017-04-01,7035.0,0.0,True,RT DanScavino POTUS realDonaldTrump signs...,4,"[trade, American manufac]",[],[],[]


In [366]:
df_tweets_aggr = df_tweets_padded.groupby('created_at', as_index = False).agg({
    'trump_tweet_cnt': 'sum',
    'trump_retweet_cnt': 'sum',
    'retweet_count': 'sum',
    'favorite_count': 'sum',
    'polarity': 'sum',
    'trade_tags_score': 'sum'
})
df_tweets_aggr.head()


,created_at,trump_tweet_cnt,trump_retweet_cnt,retweet_count,favorite_count,polarity,trade_tags_score
0,2017-01-20,13,0,417567.0,1867501.0,7.6,0
1,2017-01-21,2,1,103818.0,467521.0,5.3,0
2,2017-01-22,4,0,184236.0,949408.0,6.6,0
3,2017-01-23,1,0,26750.0,177839.0,1.4,0
4,2017-01-24,4,0,86707.0,495813.0,-1.2,0


In [367]:
df_tweets_aggr.to_csv('aggr_res.csv')